In [1]:
import torch
import numpy as np
import pandas as pd
import os
import sys

In [2]:
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score

In [3]:
from torch.utils.data import TensorDataset, DataLoader

In [4]:
import torch.nn as nn
from torch import optim

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
from sklearn.preprocessing import StandardScaler

## General split

In [112]:
data = pd.read_csv('../data/gesture-recognition-and-biometrics-electromyogram-grabmyo-1.0.2/features_split.csv')

In [113]:
data.head()

,Unnamed: 0,session,participant,gesture,index,iemg,mav,ssi,myopulse,wflen,...,kurtosis_f_w3,kurtosis_f_w4,kurtosis_f_w5,kurtosis_f_w6,kurtosis_f_w7,kurtosis_f_w8,kurtosis_f_w9,kurtosis_f_w10,kurtosis_f_w11,kurtosis_f_w12
0,0,1,1,10,0,11996.863087,0.041842,997.135823,6.224512,8143.485828,...,61.940683,116.130710,31.371157,107.176549,64.459302,241.232060,54.880235,56.951175,42.644289,118.918727
1,1,1,1,10,0,10550.150337,0.036796,1019.627142,6.443555,6646.721181,...,160.171639,87.566832,154.616093,280.595181,160.756112,98.589493,135.655396,194.396727,294.858908,537.374741
2,2,1,1,10,0,9230.366582,0.032193,630.013233,6.033789,6023.438546,...,195.666850,90.568034,94.299202,276.105575,101.698579,49.607516,150.348389,151.177847,156.860637,115.905382
3,3,1,1,10,0,15949.504704,0.055627,1995.234560,4.999609,9318.559036,...,33.886272,80.377706,46.373047,64.445165,31.711430,33.328942,50.491362,81.876212,69.070232,142.723324
4,4,1,1,10,0,15936.956936,0.055584,1830.412922,5.870117,10476.465579,...,48.630658,39.722917,25.079065,84.877336,29.229362,48.843301,44.917769,39.354396,78.705208,176.820713


In [114]:
data.columns

Index(['Unnamed: 0', 'session', 'participant', 'gesture', 'index', 'iemg',
       'mav', 'ssi', 'myopulse', 'wflen',
       ...
       'kurtosis_f_w3', 'kurtosis_f_w4', 'kurtosis_f_w5', 'kurtosis_f_w6',
       'kurtosis_f_w7', 'kurtosis_f_w8', 'kurtosis_f_w9', 'kurtosis_f_w10',
       'kurtosis_f_w11', 'kurtosis_f_w12'],
      dtype='object', length=573)

In [115]:
data.drop(columns=["Unnamed: 0"], inplace=True)

In [116]:
X = data.iloc[:, 4:].values
Y = (data.iloc[:, 2]-1).values

In [120]:
# X = scaler.fit_transform(X)

In [126]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=3443)

In [130]:
np.mean(x_train, axis = 0)[0], np.mean(x_test, axis = 0)[0]

(11869.30537947961, 11690.020190303136)

In [131]:
scaler = StandardScaler()
# scaler.fit(x_train)
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [144]:
(11869.30537947961 - scaler.mean_[0]) / scaler.scale_[0]

0.0

In [143]:
np.mean(x_train, axis = 0)[0], np.mean(x_test, axis = 0)[0]

(-3.5448661740336066e-15, -0.025975693904163086)

In [124]:
def simple_model(input_shape, output_shape):
    model = nn.Sequential(
        nn.Linear(input_shape, 32),  # Input layer: Fully connected (linear) with 64 units
        nn.ReLU(),  # Activation function: ReLU
        nn.Linear(32, 64),
        nn.ReLU(),  # Activation function: ReLU
        nn.Linear(64, output_shape)  # Output layer: Fully connected (linear) with 'output_shape' units
    )
    
    return model

In [125]:
np.mean(x_test, axis = 0)

array([-4.30960533e-03, -4.30960533e-03, -1.27448003e-02,  1.51637296e-02,
       -7.31201255e-03, -1.07725574e-02, -9.60707174e-03, -1.13731242e-02,
       -4.26524144e-03, -4.10398843e-03, -1.05209144e-02, -1.16482641e-02,
       -8.39661147e-03, -9.22285350e-03, -2.23799034e-02, -1.45045427e-02,
        1.21200481e-02, -1.04699546e-02, -9.60281236e-03, -1.48386588e-02,
       -1.07118956e-02, -1.12877531e-02, -1.50093771e-02, -9.97453521e-03,
       -8.32076348e-03, -1.19983399e-02, -8.32280337e-03, -1.44709320e-02,
        1.46963853e-02,  4.62257456e-04, -1.23439304e-02, -1.08278686e-02,
        5.09971237e-04, -1.57961928e-02,  2.80223352e-02, -1.81294651e-03,
       -1.18381888e-02, -1.21528148e-02, -1.17554037e-02, -1.57028921e-02,
       -1.97276395e-02, -2.04161153e-02, -2.80162313e-02, -2.09194462e-02,
        8.64505555e-03, -1.59181720e-02, -1.25563527e-02, -2.03507592e-02,
       -1.81027113e-02, -1.96928469e-02, -1.84992339e-02, -1.80151075e-02,
       -2.57879847e-02, -

In [84]:
train_dataset = TensorDataset(torch.tensor(x_train).type(torch.float32), torch.tensor(y_train).type(torch.LongTensor))
test_dataset = TensorDataset(torch.tensor(x_test).type(torch.float32), torch.tensor(y_test).type(torch.LongTensor))

In [85]:
len(np.unique(y_train))

17

In [86]:
# Define batch size and whether to shuffle the data
batch_size = 128
shuffle = True

# Create data loaders for training and testing
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [87]:
len(train_dataset)

12280

In [88]:
x_train.shape

(12280, 568)

In [90]:
model = simple_model(x_train.shape[1], len(np.unique(y_train)))

In [91]:
criterion = nn.CrossEntropyLoss()  
optimizer = optim.Adam(model.parameters(), lr=5e-4)

In [92]:
# Define training parameters
num_epochs = 1

In [93]:
# Training loop
for epoch in range(num_epochs):
    # Training phase
    model.train()
    total_train_loss = 0.0
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

    # Validation phase
    model.eval()
    total_val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, labels) in enumerate(test_loader):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Calculate and log metrics
    train_loss = total_train_loss / len(train_loader)
    val_loss = total_val_loss / len(test_loader)
    val_accuracy = 100 * correct / total

    print(f"Epoch [{epoch + 1}/{num_epochs}] - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")

Epoch [1/1] - Train Loss: 2.5608, Val Loss: 95683148.8650, Val Accuracy: 28.39%


In [96]:
total_val_loss

2296395572.7591124

In [98]:
x_train.shape

(12280, 568)

In [99]:
x_test.shape

(3071, 568)

In [108]:
np.mean(x_train, axis=0)

array([ 7.16023784e-15,  3.01636204e-15, -5.76628864e-16,  1.08755296e-16,
        5.21586032e-15,  5.48678909e-16,  8.53307653e-16,  1.49446145e-17,
       -2.08252706e-15,  1.38542815e-15, -1.31577476e-15, -1.39494822e-15,
        1.61962825e-15,  2.95488389e-16,  5.51486110e-16, -4.46299711e-16,
        2.46360569e-15,  3.00624075e-15,  3.83921993e-16,  9.48702752e-16,
       -8.63454937e-16, -1.57236918e-15, -2.15885489e-15,  1.54192168e-15,
        7.82811203e-16, -4.27060667e-16, -1.90170897e-16, -2.94602381e-16,
        2.68845749e-15, -9.13890751e-16, -4.36819193e-16, -5.36385540e-16,
       -4.23631904e-16, -3.45154596e-16, -3.72132654e-16,  4.27142035e-16,
       -1.56725881e-15, -4.63647624e-15,  5.60974087e-15,  3.92590412e-15,
       -4.54134558e-15,  3.87670352e-15,  3.96387592e-15, -3.48008358e-15,
        2.50669011e-15,  3.47822567e-15, -4.71223675e-15,  5.69889774e-15,
       -3.10716436e-15,  2.80744709e-15,  4.43224899e-15,  3.61261871e-15,
       -4.65710532e-16,  

In [110]:
np.mean(x_test, axis=0)

array([-4.30960533e-03, -4.30960533e-03, -1.27448003e-02,  1.51637296e-02,
       -7.31201255e-03, -1.07725574e-02, -9.60707174e-03, -1.13731242e-02,
       -4.26524144e-03, -4.10398843e-03, -1.05209144e-02, -1.16482641e-02,
       -8.39661147e-03, -9.22285350e-03, -2.23799034e-02, -1.45045427e-02,
        1.21200481e-02, -1.04699546e-02, -9.60281236e-03, -1.48386588e-02,
       -1.07118956e-02, -1.12877531e-02, -1.50093771e-02, -9.97453521e-03,
       -8.32076348e-03, -1.19983399e-02, -8.32280337e-03, -1.44709320e-02,
        1.46963853e-02,  4.62257456e-04, -1.23439304e-02, -1.08278686e-02,
        5.09971237e-04, -1.57961928e-02,  2.80223352e-02, -1.81294651e-03,
       -1.18381888e-02, -1.21528148e-02, -1.17554037e-02, -1.57028921e-02,
       -1.97276395e-02, -2.04161153e-02, -2.80162313e-02, -2.09194462e-02,
        8.64505555e-03, -1.59181720e-02, -1.25563527e-02, -2.03507592e-02,
       -1.81027113e-02, -1.96928469e-02, -1.84992339e-02, -1.80151075e-02,
       -2.57879847e-02, -

In [111]:
def predict(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    y_pred = []

    with torch.no_grad():
        for batch_idx, (inputs, labels) in enumerate(test_loader):
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            y_pred.extend(predicted.tolist())

    return y_pred

In [32]:
y_test_pred = predict(model, test_loader)

In [36]:
report = classification_report(y_test, y_test_pred)
print(report)

              precision    recall  f1-score   support

           0       0.72      0.69      0.71       159
           1       0.65      0.68      0.67       165
           2       0.78      0.77      0.78       180
           3       0.71      0.70      0.71       204
           4       0.83      0.82      0.82       186
           5       0.86      0.93      0.90       180
           6       0.75      0.71      0.73       184
           7       0.83      0.85      0.84       170
           8       0.78      0.74      0.76       192
           9       0.85      0.81      0.83       207
          10       0.96      0.96      0.96       167
          11       0.92      0.92      0.92       191
          12       0.85      0.89      0.87       149
          13       0.91      0.92      0.91       192
          14       0.80      0.89      0.84       177
          15       0.78      0.72      0.75       191
          16       0.93      0.96      0.94       177

    accuracy              

In [47]:
precision_micro = precision_score(y_test, y_test_pred, average = "micro")

In [44]:
test_parts

[39, 40, 41, 42, 43]

## Split on participants

In [12]:
participants = list(range(5,43))
test_participants = list(range(1,5))

In [13]:
train_df = data[data['participant'].isin(participants)]
test_df = data[data['participant'].isin(test_participants)]

In [14]:
train_df

,session,participant,gesture,index,iemg,mav,ssi,myopulse,wflen,diffvar,...,kurtosis_f_w3,kurtosis_f_w4,kurtosis_f_w5,kurtosis_f_w6,kurtosis_f_w7,kurtosis_f_w8,kurtosis_f_w9,kurtosis_f_w10,kurtosis_f_w11,kurtosis_f_w12
119,1,10,10,0,6797.436339,0.023708,313.982240,7.401855,5733.301325,0.000904,...,79.115045,32.591676,58.994440,54.839028,50.292229,104.307591,61.155945,196.373417,92.713179,405.962532
120,1,10,10,0,14079.062764,0.049104,1291.830907,6.844043,10385.642760,0.002613,...,12.381097,12.099599,10.995862,17.486544,14.211067,14.489695,16.576432,160.712336,79.746286,170.230168
121,1,10,10,0,11472.717905,0.040014,960.778273,6.713477,8706.221451,0.002182,...,48.730862,305.459894,212.763290,36.980016,29.056499,299.450092,55.901194,64.921149,59.408569,167.539726
122,1,10,10,0,17507.830172,0.061062,2332.784349,6.788477,13368.566593,0.005666,...,51.993926,51.528948,20.339587,43.117535,34.757809,59.816264,20.777295,59.169183,45.939629,43.873509
123,1,10,10,0,18614.528512,0.064922,2580.763916,6.342676,13226.563635,0.005325,...,306.354173,55.281935,15.773219,26.464685,84.562723,76.696436,19.323875,37.940924,24.052192,41.660051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15346,3,9,9,0,7435.557234,0.025933,369.818221,6.280762,5792.222983,0.001070,...,29.589425,25.460694,182.962449,34.957161,18.846525,37.714119,26.569683,32.614845,12.793219,16.657922
15347,3,9,9,0,10471.625678,0.036522,713.052200,6.967187,8628.380136,0.002070,...,18.226247,35.715660,113.822913,64.223311,8.214120,39.211218,29.910432,26.426457,24.892159,82.241098
15348,3,9,9,0,11279.138159,0.039339,815.440185,6.917773,9229.795138,0.002355,...,15.486525,29.417342,25.153035,19.632020,15.590478,15.462391,34.940941,37.564280,14.208021,15.811267
15349,3,9,9,0,9210.946156,0.032125,563.422838,7.359570,8040.357766,0.001839,...,14.527208,20.281059,20.418717,18.471741,14.276295,10.083487,27.337412,12.804675,20.220452,37.494222


In [15]:
x_train = train_df.iloc[:, 4:].values
y_train = (train_df.iloc[:, 2] - 1).values

x_test = test_df.iloc[:, 4:].values
y_test = (test_df.iloc[:, 2] - 1).values


In [16]:
scaler = StandardScaler()

In [17]:
scaler.fit(x_train)

StandardScaler()

In [18]:
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [19]:
x_train

array([[-0.71151404, -0.71151404, -0.44104706, ...,  1.18273398,
         0.3947441 ,  2.51393605],
       [ 0.3502168 ,  0.3502168 , -0.06034413, ...,  0.86820923,
         0.26368513,  0.75742463],
       [-0.02981323, -0.02981323, -0.18923188, ...,  0.02334703,
         0.05812778,  0.73737735],
       ...,
       [-0.05803901, -0.05803901, -0.24581592, ..., -0.21793596,
        -0.39872317, -0.39319618],
       [-0.35960121, -0.35960121, -0.34393309, ..., -0.43631153,
        -0.33795433, -0.23163008],
       [-0.30904915, -0.30904915, -0.32088124, ..., -0.36078509,
        -0.41728609, -0.21889639]])

In [20]:
x_test.shape

(1428, 568)

In [21]:
x_train.shape

(13566, 568)

In [22]:
def simple_model(input_shape, output_shape):
    model = nn.Sequential(
        nn.Linear(input_shape, 32),  # Input layer: Fully connected (linear) with 64 units
        nn.ReLU(),  # Activation function: ReLU
        nn.Linear(32, 64),
        nn.ReLU(),  # Activation function: ReLU
        nn.Linear(64, output_shape)  # Output layer: Fully connected (linear) with 'output_shape' units
    )
    
    return model

In [23]:
train_dataset = TensorDataset(torch.tensor(x_train).type(torch.float32), torch.tensor(y_train).type(torch.LongTensor))
test_dataset = TensorDataset(torch.tensor(x_test).type(torch.float32), torch.tensor(y_test).type(torch.LongTensor))

In [24]:
len(np.unique(y_train))

17

In [25]:
# Define batch size and whether to shuffle the data
batch_size = 128
shuffle = True

# Create data loaders for training and testing
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [26]:
model = simple_model(x_train.shape[1], len(np.unique(y_train)))

In [27]:
criterion = nn.CrossEntropyLoss()  
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [28]:
# Define training parameters
num_epochs = 100

In [68]:
len(test_loader)

12

In [74]:
next(iter(test_loader))

[tensor([[ 0.0466,  0.0466, -0.1751,  ..., -0.0469, -0.1113,  0.3751],
         [-0.1643, -0.1643, -0.1663,  ...,  1.1653,  2.4379,  3.4931],
         [-0.3568, -0.3568, -0.3180,  ...,  0.7841,  1.0431,  0.3526],
         ...,
         [-0.3627, -0.3627, -0.2277,  ...,  0.1066,  0.2954,  0.2957],
         [ 1.9771,  1.9771,  1.6013,  ..., -0.2454, -0.2063, -0.1831],
         [ 2.0838,  2.0838,  2.2605,  ...,  0.4058,  0.3251,  0.2316]]),
 tensor([ 9,  9,  9,  9,  9,  9,  9, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11,
         11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 14,
         14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16,
         16, 16,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  2,  2,
          2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,  4,  4,  4,
          4,  5,  5,  5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  6,  6,  7,  7,  7,
          7,  7,  7,  7,  8,  8,  8,  8,  8,  8,  8,  9,  9,  9,  9,  9,

In [30]:
# Training loop
for epoch in range(num_epochs):
    # Training phase
    model.train()
    total_train_loss = 0.0
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

    # Validation phase
    model.eval()
    total_val_loss = 0.0
    print(total_val_loss)
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, labels) in enumerate(test_loader):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Calculate and log metrics
    train_loss = total_train_loss / len(train_loader)
    val_loss = total_val_loss / len(test_loader)
    val_accuracy = 100 * correct / total

    print(f"Epoch [{epoch + 1}/{num_epochs}] - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")

Epoch [1/100] - Train Loss: 0.0609, Val Loss: 5.1502, Val Accuracy: 60.15%
Epoch [2/100] - Train Loss: 0.0669, Val Loss: 5.3280, Val Accuracy: 59.38%
Epoch [3/100] - Train Loss: 0.0637, Val Loss: 5.3819, Val Accuracy: 59.45%
Epoch [4/100] - Train Loss: 0.0502, Val Loss: 5.4860, Val Accuracy: 60.22%
Epoch [5/100] - Train Loss: 0.0521, Val Loss: 5.4284, Val Accuracy: 59.73%
Epoch [6/100] - Train Loss: 0.0549, Val Loss: 5.4382, Val Accuracy: 60.50%
Epoch [7/100] - Train Loss: 0.0659, Val Loss: 5.4062, Val Accuracy: 59.52%
Epoch [8/100] - Train Loss: 0.0739, Val Loss: 5.7117, Val Accuracy: 58.96%
Epoch [9/100] - Train Loss: 0.0774, Val Loss: 5.5320, Val Accuracy: 60.29%
Epoch [10/100] - Train Loss: 0.0575, Val Loss: 5.4719, Val Accuracy: 60.85%
Epoch [11/100] - Train Loss: 0.0552, Val Loss: 5.5890, Val Accuracy: 60.50%
Epoch [12/100] - Train Loss: 0.0455, Val Loss: 5.6710, Val Accuracy: 59.80%
Epoch [13/100] - Train Loss: 0.0605, Val Loss: 5.6944, Val Accuracy: 60.22%
Epoch [14/100] - Trai

In [509]:
y_test_pred = predict(model, test_loader)

In [510]:
y_test_pred

[9,
 9,
 9,
 9,
 9,
 9,
 9,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 13,
 13,
 12,
 13,
 13,
 13,
 13,
 14,
 9,
 14,
 14,
 14,
 14,
 14,
 4,
 15,
 15,
 0,
 15,
 15,
 15,
 16,
 12,
 16,
 16,
 16,
 16,
 16,
 0,
 0,
 15,
 0,
 0,
 15,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 12,
 1,
 2,
 2,
 2,
 6,
 6,
 2,
 6,
 12,
 12,
 1,
 6,
 8,
 0,
 8,
 4,
 8,
 4,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 6,
 6,
 6,
 12,
 6,
 6,
 6,
 7,
 7,
 7,
 5,
 5,
 7,
 5,
 8,
 0,
 8,
 0,
 8,
 8,
 8,
 9,
 9,
 12,
 9,
 0,
 9,
 12,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 12,
 12,
 11,
 11,
 11,
 12,
 11,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 13,
 13,
 13,
 13,
 12,
 12,
 12,
 4,
 14,
 14,
 4,
 14,
 14,
 4,
 1,
 15,
 0,
 0,
 15,
 15,
 15,
 12,
 16,
 13,
 13,
 13,
 12,
 13,
 0,
 0,
 16,
 0,
 15,
 15,
 15,
 2,
 1,
 1,
 1,
 1,
 12,
 12,
 2,
 2,
 10,
 2,
 2,
 2,
 2,
 13,
 13,
 13,
 13,
 7,
 7,
 13,
 8,
 11,
 4,
 4,
 4,
 11,
 4,
 5,
 1,
 1,
 13,
 1,
 1,
 5,
 2,
 2,
 2,
 2,
 2,


In [511]:
y_test

array([9, 9, 9, ..., 8, 8, 8], dtype=int64)

In [512]:
report = classification_report(y_test, y_test_pred)
print(report)

              precision    recall  f1-score   support

           0       0.53      0.61      0.57        84
           1       0.49      0.43      0.46        84
           2       0.52      0.57      0.55        84
           3       0.30      0.27      0.29        84
           4       0.72      0.50      0.59        84
           5       0.79      0.60      0.68        84
           6       0.37      0.55      0.44        84
           7       0.64      0.40      0.50        84
           8       0.69      0.52      0.59        84
           9       0.79      0.45      0.58        84
          10       0.83      0.89      0.86        84
          11       0.88      0.93      0.90        84
          12       0.47      0.74      0.58        84
          13       0.66      0.74      0.70        84
          14       0.52      0.79      0.63        84
          15       0.66      0.46      0.55        84
          16       0.88      0.92      0.90        84

    accuracy              

### Split by session

In [467]:
participants = [1,3]
test_participants = [2]

In [468]:
train_df = data[data['session'].isin(participants)]
test_df = data[data['session'].isin(test_participants)]

In [469]:
train_df

,session,participant,gesture,index,iemg,mav,ssi,myopulse,wflen,diffvar,...,kurtosis_f_w3,kurtosis_f_w4,kurtosis_f_w5,kurtosis_f_w6,kurtosis_f_w7,kurtosis_f_w8,kurtosis_f_w9,kurtosis_f_w10,kurtosis_f_w11,kurtosis_f_w12
0,1,1,10,0,11996.863087,0.041842,997.135823,6.224512,8143.485828,0.001888,...,61.940683,116.130710,31.371157,107.176549,64.459302,241.232060,54.880235,56.951175,42.644289,118.918727
1,1,1,10,0,10550.150337,0.036796,1019.627142,6.443555,6646.721181,0.001438,...,160.171639,87.566832,154.616093,280.595181,160.756112,98.589493,135.655396,194.396727,294.858908,537.374741
2,1,1,10,0,9230.366582,0.032193,630.013233,6.033789,6023.438546,0.001169,...,195.666850,90.568034,94.299202,276.105575,101.698579,49.607516,150.348389,151.177847,156.860637,115.905382
3,1,1,10,0,15949.504704,0.055627,1995.234560,4.999609,9318.559036,0.003030,...,33.886272,80.377706,46.373047,64.445165,31.711430,33.328942,50.491362,81.876212,69.070232,142.723324
4,1,1,10,0,15936.956936,0.055584,1830.412922,5.870117,10476.465579,0.003688,...,48.630658,39.722917,25.079065,84.877336,29.229362,48.843301,44.917769,39.354396,78.705208,176.820713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15346,3,9,9,0,7435.557234,0.025933,369.818221,6.280762,5792.222983,0.001070,...,29.589425,25.460694,182.962449,34.957161,18.846525,37.714119,26.569683,32.614845,12.793219,16.657922
15347,3,9,9,0,10471.625678,0.036522,713.052200,6.967187,8628.380136,0.002070,...,18.226247,35.715660,113.822913,64.223311,8.214120,39.211218,29.910432,26.426457,24.892159,82.241098
15348,3,9,9,0,11279.138159,0.039339,815.440185,6.917773,9229.795138,0.002355,...,15.486525,29.417342,25.153035,19.632020,15.590478,15.462391,34.940941,37.564280,14.208021,15.811267
15349,3,9,9,0,9210.946156,0.032125,563.422838,7.359570,8040.357766,0.001839,...,14.527208,20.281059,20.418717,18.471741,14.276295,10.083487,27.337412,12.804675,20.220452,37.494222


In [470]:
x_train = train_df.iloc[:, 4:].values
y_train = (train_df.iloc[:, 2] - 1).values

x_test = test_df.iloc[:, 4:].values
y_test = (test_df.iloc[:, 2] - 1).values


In [471]:
scaler = StandardScaler()

In [472]:
scaler.fit(x_train)

StandardScaler()

In [473]:
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [474]:
def simple_model(input_shape, output_shape):
    model = nn.Sequential(
        nn.Linear(input_shape, 32),  # Input layer: Fully connected (linear) with 64 units
        nn.ReLU(),  # Activation function: ReLU
        nn.Linear(32, 64),
        nn.ReLU(),  # Activation function: ReLU
        nn.Linear(64, output_shape)  # Output layer: Fully connected (linear) with 'output_shape' units
    )
    
    return model

In [475]:
train_dataset = TensorDataset(torch.tensor(x_train).type(torch.float32), torch.tensor(y_train).type(torch.LongTensor))
test_dataset = TensorDataset(torch.tensor(x_test).type(torch.float32), torch.tensor(y_test).type(torch.LongTensor))

In [476]:
len(np.unique(y_train))

17

In [477]:
# Define batch size and whether to shuffle the data
batch_size = 128
shuffle = True

# Create data loaders for training and testing
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [478]:
model = simple_model(x_train.shape[1], len(np.unique(y_train)))

In [479]:
criterion = nn.CrossEntropyLoss()  
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [480]:
# Define training parameters
num_epochs = 100

In [481]:
# Training loop
for epoch in range(num_epochs):
    # Training phase
    model.train()
    total_train_loss = 0.0
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

    # Validation phase
    model.eval()
    total_val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, labels) in enumerate(test_loader):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Calculate and log metrics
    train_loss = total_train_loss / len(train_loader)
    val_loss = total_val_loss / len(test_loader)
    val_accuracy = 100 * correct / total

    print(f"Epoch [{epoch + 1}/{num_epochs}] - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")

Epoch [1/100] - Train Loss: 2.4191, Val Loss: 1365127.5563, Val Accuracy: 38.38%
Epoch [2/100] - Train Loss: 1.5826, Val Loss: 9924327.7019, Val Accuracy: 58.84%
Epoch [3/100] - Train Loss: 1.1271, Val Loss: 21853153.0808, Val Accuracy: 65.88%
Epoch [4/100] - Train Loss: 0.9310, Val Loss: 27828983.3690, Val Accuracy: 69.55%
Epoch [5/100] - Train Loss: 0.8197, Val Loss: 34200506.5026, Val Accuracy: 72.11%
Epoch [6/100] - Train Loss: 0.7383, Val Loss: 36717312.8797, Val Accuracy: 72.52%
Epoch [7/100] - Train Loss: 0.6748, Val Loss: 37467072.8526, Val Accuracy: 73.99%
Epoch [8/100] - Train Loss: 0.6270, Val Loss: 47859888.8476, Val Accuracy: 74.42%
Epoch [9/100] - Train Loss: 0.5921, Val Loss: 45645607.2486, Val Accuracy: 74.52%
Epoch [10/100] - Train Loss: 0.5652, Val Loss: 42729972.0853, Val Accuracy: 75.20%
Epoch [11/100] - Train Loss: 0.5350, Val Loss: 38915735.2525, Val Accuracy: 75.83%
Epoch [12/100] - Train Loss: 0.5119, Val Loss: 47672640.8599, Val Accuracy: 77.14%
Epoch [13/100] 

Epoch [100/100] - Train Loss: 0.0618, Val Loss: 299416731.6430, Val Accuracy: 76.76%


In [486]:
y_test_pred = predict(model, test_loader)

In [487]:
y_test

array([9, 9, 9, ..., 8, 8, 8], dtype=int64)

In [488]:
report = classification_report(y_test, y_test_pred)
print(report)

              precision    recall  f1-score   support

           0       0.65      0.61      0.63       301
           1       0.65      0.51      0.57       301
           2       0.69      0.72      0.71       301
           3       0.57      0.64      0.61       301
           4       0.74      0.80      0.77       301
           5       0.88      0.87      0.87       301
           6       0.66      0.65      0.66       301
           7       0.78      0.85      0.81       301
           8       0.73      0.72      0.72       301
           9       0.79      0.82      0.80       301
          10       0.95      0.92      0.93       301
          11       0.86      0.85      0.86       301
          12       0.81      0.82      0.82       301
          13       0.84      0.83      0.84       301
          14       0.82      0.79      0.81       301
          15       0.70      0.66      0.68       301
          16       0.93      0.96      0.94       301

    accuracy              

In [ ]:
re